In [66]:
from typing import Any, List, Dict

# Tutorial 2
## Shannon codes

The year is 2148. In this future, the world has changed. Few things of the past are now remembered. The technocracy has taken over and our access to information has become fully controlled and obfuscated. All we know, is what we are told, and we are told very little.

You are part of the Resistance, whose purpose is to Liberate Information. The Resistance follows the percepts of a very important manifesto, known as ["The Mathematical Theory of Communication"](https://archive.org/details/ost-engineering-shannon1948), a 200-year old text which contains the necessary and sufficient knowledge to setup communication systems, which has been recovered from the vaults of the corporations at great costs. Your goal is to share this text with other Resistance members everywhere in the world by broadcasting the text, in a loop, until the heat death of the universe.

The only thing that's left is to encode it. You must use a code that is easy to revert. Someone suggested Morse code, but this is too inefficient: you have to add punctuation or spaces to help decoding. This means that no codeword should be the prefix of another.  A code with these properties is defined in the very text you must share:


> Arrange the messages of length N in order of decreasing probability and suppose their probabilities are
$p_1 \le p_2 \le  ... p_n$. Let $P_s = \sum_1^{s-1} p_i$; that is $P_s$ is the cumulative probability up to, but not including, $p_s$.
We first encode into a binary system. The binary code for message s is obtained by expanding $P_s$ as a binary number. The expansion is carried out to $m_s$ places, where $m_s$ is the integer satisfying:
$$log_2 \frac{1}{p_s} \le m_s \le 1 + \log_2 \frac{1}{p_s}$$
Thus the messages of high probability are represented by short codes and those of low probability by long codes.

## Your task
Members of the Resistance already have transmitters and receivers, but don't know how to use them. Your task is to help them encode and decode "The Mathematical Theory of Communication" using the Shannon code defined above. Suppose the alphabet is the alphanumerics (all lowercase): to make it easier, the function `infinite_shannon()` loads the paper in plain text for you and removes all non-alphanumeric characters.


#### Bonus
For the purpose of this exercise, we ignore mathematical symbols, but you may try including them - how much bigger does the code get?


In [61]:
from collections import Counter, OrderedDict
from math import log2, ceil

In [60]:
alphabet = "abcdefghijklmnopqrstuvwxyz01234567890"

In [52]:
def infinite_shannon() -> str:

    text = ''
    with open('shannon1948.txt', 'r') as f:
        text = f.read()
    text = text.lower()
    text = [ t for t in text if t in alphabet ]
    return text

text = infinite_shannon()

### Step 1: Cumulative probability
First step is to obtain the probability distributions of each symbol from the text, and then compute the cumulative probability distribution. For convenience, you should first re-order the symbols by their frequency (most frequent first, by using `sorted(list, reverse=True)`), or by using an `OrderedDict`. You may want to plot the cumulative distribution to make sure you obtain the discrete staircase shape.

Hint: Frequency counting can be easily done in 'vanilla' Python using `collections.Counter`. You don't have to, but you can use `cumsum` in `numpy` to compute cumulative distributions.

In [72]:
def get_freqs(text: str) -> Dict[chr, int]:
    raise NotImplementedError

def cum_prob(freqs: Dict[chr, int]) -> Dict[chr, float]:
    raise NotImplementedError

### Step 2: Binary encoding
Now use binary encoding to encode the cumulative probability, $F(x_i)$ (or $P_i$ in Shannon's notation), then take the first $\ell(x_i) = \lceil \frac{1}{p(x_i)} \rceil$ digits (or $m_s$) from it to obtain the codeword. 

If you haven't seen this before, binary encoding can be done for non-integers too! For example:

- 0.1 = one half
- 0.01 = one fourth
- 0.001 = one eighth
- 0.101 = five eighths
- 0.1111 = fifteen sixteenths

Hint: If encoding a decimal integer into binary is done by iteratively dividing by 2 repeatedly and taking the remainders until the quotient remains 0, encoding the decimal fractional is done by iteratively multiplying the fractional part by 2 and taking the integral part of the result, until the fractional part remains 0. 

It's easiest to operate with strings, and to build a dictionary that returns a codeword for each character in the alphabet, but implementation details are up to you.

In [108]:
def get_binary(f: float) -> str:
    raise NotImplementedError

def get_shannon_codeword(p: float, f: float) -> str:
    raise NotImplementedError

def shannon_code_dict(freqs: Dict[chr, int]) -> Dict[chr, str]:
    f = cum_prob(freqs)
    
    raise NotImplementedError

Now to put it all together! The function below is meant to extract the probabilities from the given text, then produce a code dictionary that you can use to encode (and decode!) the text.

Alternatively, you could pass in a code dictionary, and encode a (possibly different) text.

In [117]:
def shannon_encode(text: str) -> str:
    freqs = get_freqs(text)
    codes = shannon_code_dict(freqs)
    
    return [ codes[x] for x in text ]

In [111]:
def encode(text: str, codes: Dict[chr, str]) -> str:
    
    return [ codes[x] for x in text ]

### Step 3: Binary decoding

Now you must help the other members of the resistance with the decoding. Given a sequence in the Shannon code, can you write a function to decode it back to English?

In [113]:
def decode(coded: str, codes: Dict[chr, str]) -> str:
    raise NotImplementedError

### Examples 

You may want to test your implementation first. Here is an example from Shannon's paper.

In [68]:
def dictify(keys: List[Any], values: List[Any]) -> Dict[Any, Any]:
    return dict(zip(keys, values))

In [69]:
def example():
    alph = [ 'a', 'b', 'c', 'd' ]
    prob = [ 1/2, 1/4, 1/8, 1/8 ]
    code = [ '0', '10', '110', '111' ] 
    
    return dictify(alph, prob), dictify(alph, code)

And here is someting to decode! 
c = `0110010011001001100100110010`

Try both using the dictionary from `example()` or using the `c` string as text, deriving probabilities from it. How do the codes differ?

In [89]:
c = '0110010011001001100100110010'

### Testing your code
A few sanity checks... you want to check all codewords are different from each other, that none of them is the prefix of another, and that the length of the codes is higher than the entropy. You can apply the functions below to the list of unique codewords to check.

In [120]:
def __unique(codes: List[str]) -> bool:
    
    return sorted(codes) == sorted(set(codes))

def __prefix_free(codes: List[str]) -> bool:
    
    for c in codes:
        if any([d.startswith(c) for d in codes]):
            return False
        
    return True

def __entropy(probs: List[str], codes: List[str]) -> bool:
    
    H = sum( -p * log2(p) for p in probs )
    print(f"Entropy of the distribution is: {H}")
    L = 1/len(codes) * sum( len(c) for c in codes)
    print(f"Mean code length: {L}")
    
    return H < L

And now, are you ready to encode the Shannon text?

In [ ]:
freqs = get_freqs(text)
codes = code_dict(freqs)
coded = encode(text, codes)

text2 = decode(coded, codes)

Does the decoded text match the original?

In [ ]:
text == text2

## Huffman code

The year is 2152. You succeeded in your mission, and the Resistance's communications and research has started growing exponentially. Now many more codes have been unearthed and are being reimplemented. Developments in Free Information Theory have revealed the most efficient form of a symbol code is what is called the Huffman code. As a hero of the Resistance, you've been asked to update the beacon and send the Shannon paper this time in the Huffman code.

The Huffman code, recovered from a lost scroll from 200 years ago called ["A Method for the Construction of Minimum-Redundancy Codes"](http://compression.ru/download/articles/huff/huffman_1952_minimum-redundancy-codes.pdf), works as follows:

1. Pick the last 2 probable symbols
2. Combine them into a single symbol, forming a binary tree
3. If more than one symbol remains, go to step 1.


### Step 1: build tree
The tree is built incrementally from the symbols, starting with the least frequent. You can use the same `get_freqs` from the Shannon code (Huffman encoding doesn't care if you pass in probabilities or frequencies, but make sure they are storted).

To build and traverse  trees, you may find a node data structure useful, such as the one below.

The nodes either have children, or they are a leaf, thus having a label that represents the string being encoded. A link to the parent node can help with traversing it up and down.

In [105]:
class Node():
    def __init__(self, l: 'Node' = None, r: 'Node' = None,
                       p: 'Node '= None, lab: str = ''):
        self.l = l
        self.r = r
        self.p = p
        self.lab = lab

    def children(self):
        return self.l, self.r
    
    def parent(self):
        return self.p
    
    def __str__(self):
        if self.lab:
            return self.lab
        else:
            return f"(L: {str(self.l)}, R: {str(self.r)})"

For example, this tree

![](huffman-tree-example1.png)

could be expressed as (ignoring parents):

In [106]:
root = Node(
    l = Node(
            l = Node(lab = 'a'),
            r = Node(
                l = Node(lab = 'd'),
                r = Node(lab = 'e')
            )
        ),
    r = Node(
            l = Node(lab = 'b'),
            r = Node(lab = 'c')
        )
    )

print(root)

(L: (L: a, R: (L: d, R: e)), R: (L: b, R: c))


After building the tree, you can either obtain a root node with links to the children, as in the example above, or you can have a list of leaves for each codewords, with links to the parents.

Now it's time for you to build the Huffman code tree. There are many ways of doing this, but you may find the following steps useful:
1. Create one leaf node for each symbol in the alphabet, and sort these nodes in a list according to the probability of their corresponding label.
2. Create a new node that acts as the parent of the two nodes with least probability, and assign to the new node the sum of those two probabilities.
3. Add the new node to the list and remove the two nodes that have just been merged.
4. Repeat from step 2 as necessary.

Go on, now. The Resistance needs you!

In [107]:
def build_tree(freqs: Dict[chr, float]) -> 'Node':
    raise NotImplementedError

### Step 2: find code

The code can be constructed by 'traversing' the tree. Depending on your implementation, one simple way to do this would be to start at the root, and descend recursively until you reach the leaf, in a way similar to the `__str__` function the node uses to print the tree. The code will contain a `0` if the symbol is in the left subtree, and a `1` if the symbol is in the right subtree.

Regardless of your implementation, you likely want to produce a dictionary that maps each symbol to a codeword, similar to the Shannon code.

In [114]:
def huffman_code_dict(tree: 'Node') -> Dict[chr, str]:
    raise NotImplementedError  

Now in a similar way to the Shannon code, we put it all together!

In [116]:
def huffman_encode(text: str) -> str:
    freqs = get_freqs(text)
    tree  = build_tree(freqs)
    codes = huffman_code_dict(tree)
    
    return [ codes[x] for x in text ]

Once you obtain the dictionary of codes, you should be able to use  the same `encode` and `decode` functions from the Shannon codes, by passing them a text and a code dictionary. 

### Examples 

You may want to test your implementation first. Here is an example:

In [69]:
def example1():
    alph = [ 'a', 'b', 'c', 'd', 'e' ]
    prob = [ 1/4, 1/4, 1/5, 3/20, 3/20 ]
    code = [ '00', '10', '11', '010', '011' ] 
    
    return dictify(alph, prob), dictify(alph, code)

And another, taken directly from Huffman's paper:

In [85]:
def example2():
    alph = range(1, 14)
    prob = [ 0.2, 0.18, 0.1, 0.1, 0.1, 0.05, 0.06, 0.04, 0.04, 0.04, 0.04, 0.03, 0.01 ]
    code = [ '10', '000', '011', '110', '111', '0101', '00100', 
            '00101', '01000', '01001', '00110', '001110', '001111']
    
    return dictify(alph, prob), dictify(alph, code)

You should, of course, use the sanity check functions `__unique` and `__prefix_free`.

_How does the entropy check change for the Huffman code?_


You may notice that the code your algorithm produced has all the necessary properties but **is not the same** as the one in the example. Don't despair! You have just discovered that the optimal Huffman encoding is not always unique! 

_Don't forget to encode the Shannon paper! How much more efficient is the Huffman code?_